# go-数据库操作

### MySQL

Go中支持MySQL的驱动目前比较多，有如下几种，有些是支持database/sql标准，而有些是采用了自己的实现接口,常用的有如下几种:

(https://github.com/go-sql-driver/mysql)[https://github.com/go-sql-driver/mysql] 支持database/sql，全部采用go写。
(https://github.com/ziutek/mymysql)[https://github.com/ziutek/mymysql] 支持database/sql，也支持自定义的接口，全部采用go写。
(https://github.com/Philio/GoMySQL)[https://github.com/Philio/GoMySQL] 不支持database/sql，自定义接口，全部采用go写。
接下来的例子主要以第一个驱动为例，也推荐大家采用它，主要理由：

* 这个驱动比较新，维护的比较好
* 完全支持database/sql接口
* 支持keepalive，保持长连接,这个从底层就支持了keepalive。

```
CREATE TABLE `userinfo` (
	`uid` INT(10) NOT NULL AUTO_INCREMENT,
	`username` VARCHAR(64) NULL DEFAULT NULL,
	`department` VARCHAR(64) NULL DEFAULT NULL,
	`created` DATE NULL DEFAULT NULL,
	PRIMARY KEY (`uid`)
);

CREATE TABLE `userdetail` (
	`uid` INT(10) NOT NULL DEFAULT '0',
	`intro` TEXT NULL,
	`profile` TEXT NULL,
	PRIMARY KEY (`uid`)
)
```

```
package main

import (
	"database/sql"
	"fmt"
	_ "github.com/go-sql-driver/mysql"
)

func main() {
	db, err := sql.Open("mysql", "astaxie:astaxie@/test?charset=utf8")
	checkErr(err)

	//插入数据
	stmt, err := db.Prepare("INSERT userinfo SET username=?,department=?,created=?")
	checkErr(err)

	res, err := stmt.Exec("astaxie", "研发部门", "2012-12-09")
	checkErr(err)

	id, err := res.LastInsertId()
	checkErr(err)

	fmt.Println(id)
    
	//更新数据
	stmt, err = db.Prepare("update userinfo set username=? where uid=?")
	checkErr(err)

	res, err = stmt.Exec("astaxieupdate", id)
	checkErr(err)

	affect, err := res.RowsAffected()
	checkErr(err)

	fmt.Println(affect)

	//查询数据
	rows, err := db.Query("SELECT * FROM userinfo")
	checkErr(err)

	for rows.Next() {
		var uid int
		var username string
		var department string
		var created string
		err = rows.Scan(&uid, &username, &department, &created)
		checkErr(err)
		fmt.Println(uid)
		fmt.Println(username)
		fmt.Println(department)
		fmt.Println(created)
	}

	//删除数据
	stmt, err = db.Prepare("delete from userinfo where uid=?")
	checkErr(err)

	res, err = stmt.Exec(id)
	checkErr(err)

	affect, err = res.RowsAffected()
	checkErr(err)

	fmt.Println(affect)

	db.Close()

}

func checkErr(err error) {
	if err != nil {
		panic(err)
	}
}
```

* sql.Open()函数用来打开一个注册过的数据库驱动，go-sql-driver中注册了mysql这个数据库驱动，第二个参数是DSN(Data Source Name)，它是go-sql-driver定义的一些数据库链接和配置信息。它支持如下格式：
```
user@unix(/path/to/socket)/dbname?charset=utf8
user:password@tcp(localhost:5555)/dbname?charset=utf8
user:password@/dbname
user:password@tcp([de:ad:be:ef::ca:fe]:80)/dbname
```
* db.Prepare()函数用来返回准备要执行的sql操作，然后返回准备完毕的执行状态。

* db.Query()函数用来直接执行Sql返回Rows结果。

* stmt.Exec()函数用来执行stmt准备好的SQL语句

可以看到我们传入的参数都是=?对应的数据，这样做的方式可以一定程度上防止SQL注入。

### sqlite

SQLite 是一个开源的嵌入式关系数据库，实现自包容、零配置、支持事务的SQL数据库引擎。SQLite可以是说开源的Access。

SQLite 的安装和运行非常简单，在大多数情况下,只要确保SQLite的二进制文件存在即可开始创建、连接和使用数据库。

特点: 高度便携、使用方便、结构紧凑、高效、可靠。 

驱动:

Go支持sqlite的驱动也比较多，但是好多都是不支持database/sql接口的

* `https://github.com/mattn/go-sqlite3` 支持database/sql接口，基于cgo(关于cgo的知识请参看官方文档或者本书后面的章节)写的
* `https://github.com/feyeleanor/gosqlite3` 不支持database/sql接口，基于cgo写的
* `https://github.com/phf/go-sqlite3` 不支持database/sql接口，基于cgo写的
目前支持database/sql的SQLite数据库驱动只有第一个，一般采用它来开发项目的。采用标准接口有利于以后出现更好的驱动的时候做迁移。

sqlite和MySQL例子里面的代码几乎是一模一样的，唯一改变的就是导入的驱动改变了，然后调用sql.Open是采用了SQLite的方式打开。
`db, err := sql.Open("sqlite3", "./foo.db")`

### postgreSQL

PostgreSQL 是一个自由的对象-关系数据库服务器(数据库管理系统)，它在灵活的 BSD-风格许可证下发行。

PostgreSQL和MySQL比较，它更加庞大一点，因为它是用来替代Oracle而设计的。所以在企业应用中采用PostgreSQL是一个明智的选择。

MySQL被Oracle收购之后正在逐步封闭（自MySQL 5.5.31以后所有版本不再遵循GPL协议），将来也许选择PostgreSQL而不是MySQL作为项目的后端数据库。

驱动: Go实现的支持PostgreSQL的驱动也很多，因为国外很多人在开发中使用了这个数据库。

* `https://github.com/lib/pq` 支持database/sql驱动，纯Go写的
* `https://github.com/jbarham/gopgsqldriver` 支持database/sql驱动，纯Go写的
* `https://github.com/lxn/go-pgsql` 支持database/sql驱动，纯Go写的
在下面的示例中采用了第一个驱动，因为它目前使用的人最多，在github上也比较活跃。

* PostgreSQL通过`$1`,`$2`这种方式来指定要传递的参数，而不是MySQL中的?
* 在sql.Open中的dsn信息的格式也与MySQL的驱动中的dsn格式不一样，使用时请注意它们的差异。
* pg不支持LastInsertId函数，因为PostgreSQL内部没有实现类似MySQL的自增ID返回，其他的代码几乎是一模一样。

`db, err := sql.Open("postgres", "user=astaxie password=astaxie dbname=test sslmode=disable")`

### beego orm

beego orm是支持database/sql标准接口的ORM库，理论上来说，只要数据库驱动支持database/sql接口就可以无缝的接入beego orm。

目前我测试过的驱动包括下面几个：
* Mysql: github/go-mysql-driver/mysql
* PostgreSQL: github.com/lib/pq
* SQLite: github.com/mattn/go-sqlite3
* Mysql: github.com/ziutek/mymysql/godrv

暂未支持数据库:
* MsSql: github.com/denisenkom/go-mssqldb
* MS ADODB: github.com/mattn/go-adodb
* Oracle: github.com/mattn/go-oci8
* ODBC: bitbucket.org/miquella/mgodbc

安装: beego orm支持go get方式安装，是完全按照Go Style的方式来实现的。

`go get github.com/astaxie/beego`

初始化:
```
import (
	"database/sql"
	"github.com/astaxie/beego/orm"
	_ "github.com/go-sql-driver/mysql"
)

func init() {
	//注册驱动
	orm.RegisterDriver("mysql", orm.DR_MySQL)
	//设置默认数据库
	orm.RegisterDataBase("default", "mysql", "root:root@/my_db?charset=utf8", 30)
	//注册定义的model
    	orm.RegisterModel(new(User))

   	// 创建table
        orm.RunSyncdb("default", false, true)
}
```

PostgreSQL 配置:
```
//导入驱动
// _ "github.com/lib/pq"

// 注册驱动
orm.RegisterDriver("postgres", orm.DR_Postgres) 

// 设置默认数据库
//PostgresQL用户：postgres ，密码：zxxx ， 数据库名称：test ， 数据库别名：default
orm.RegisterDataBase("default", "postgres", "user=postgres password=zxxx dbname=test host=127.0.0.1 port=5432 sslmode=disable")
```

MySQL 配置:
```
//导入驱动
//_ "github.com/go-sql-driver/mysql"

//注册驱动
orm.RegisterDriver("mysql", orm.DR_MySQL)

// 设置默认数据库
//mysql用户：root ，密码：zxxx ， 数据库名称：test ， 数据库别名：default
 orm.RegisterDataBase("default", "mysql", "root:zxxx@/test?charset=utf8")
 ```
 
Sqlite 配置:
```
//导入驱动
//_ "github.com/mattn/go-sqlite3"

//注册驱动
orm.RegisterDriver("sqlite", orm.DR_Sqlite)

// 设置默认数据库
//数据库存放位置：./datas/test.db ， 数据库别名：default
orm.RegisterDataBase("default", "sqlite3", "./datas/test.db")
```

导入必须的package之后,我们需要打开到数据库的链接，然后创建一个beego orm对象（以MySQL为例)，如下所示 beego orm:
```
func main() {
    	o := orm.NewOrm()
}
```

```
SetMaxIdleConns: 根据数据库的别名，设置数据库的最大空闲连接

orm.SetMaxIdleConns("default", 30)

SetMaxOpenConns: 根据数据库的别名，设置数据库的最大数据库连接 (go >= 1.2)

orm.SetMaxOpenConns("default", 30): 目前beego orm支持打印调试，你可以通过如下的代码实现调试

orm.Debug = true
```
注意: beego orm针对驼峰命名会自动转化成下划线字段，如定义Struct名字为UserInfo，转化成底层实现时是user_info，字段命名也遵循该规则。

### redis

redis是key-value存储系统。和Memcached类似，支持存储的value类型相对更多，包括string(字符串)、list(链表)、set(集合)和zset(有序集合)。

目前应用redis最广泛的应该是新浪微博平台，其次还有Facebook收购的图片社交网站instagram。以及其他一些有名的互联网企业

Go目前支持redis的驱动有如下

* `https://github.com/garyburd/redigo` (推荐)
* `https://github.com/go-redis/redis`
* `https://github.com/hoisie/redis`
* `https://github.com/alphazero/Go-Redis`
* `https://github.com/simonz05/godis`

### mongodb

MongoDB是一个高性能，开源，无模式的文档型数据库，是一个介于关系数据库和非关系数据库之间的产品，是非关系数据库当中功能最丰富，最像关系数据库的。他支持的数据结构非常松散，采用的是类似json的bjson格式来存储数据，因此可以存储比较复杂的数据类型。

MongoDB最大的特点是支持的查询语言非常强大，语法类似面向对象的查询语言，几乎实现类似关系数据库单表查询的绝大部分功能，且还支建立索引。

目前Go支持mongoDB最好的驱动就是mgo，这个驱动目前最有可能成为官方的pkg。

安装mgo: `go get gopkg.in/mgo.v2`